## 💡展开说说 Embeddings

1. 未来的开发中，我们会频繁用到Embedding
2. 不止是基于大模型的应用，其它场景中Embedding也非常常用，比如「聚类」
3. Transformer的输入就是Embedding，其隐层也可以当做Embedding来用

## 什么是Embeddings

### 从词向量说起

从字面本身计算语义相关性是不够的
- 不同字，同义：「快乐」vs.「高兴」
- 同字，不同义：「上马」vs.「马上」

所以我们需要一种方法，能够有效计算词与词之间的关系，词向量（Word Embedding）应运而生

<img src="word2vec.png" style="margin-left: 0px" width=500px>
<img src="word2vec2.png" style="margin-left: 0px" width=500px>

<br/>

### 词向量的基本原理：用一个词上下文窗口表示它自身

<br/>
<img src="w2v.png" style="margin-left: 0px" width=500px>

### 词向量的不足

- 同一个词在不同上下文中语义不同：我从「马上」下来 vs. 我「马上」下来

### 基于整个句子，表示句中每个词，那么同时我们也就表示了整个句子

<br />
<img src="mlm.png" style="margin-left: 0px" width=500px>
<br />

### 所以，句子、篇章都可以向量化

<br />
<img src="SemanticSearch.png" style="margin-left: 0px" width=500px>

### 向量相似度计算

In [12]:
import numpy as np
from numpy import dot
from numpy.linalg import norm
from langchain.embeddings import OpenAIEmbeddings

def cos_sim(a,b):
    return dot(a, b)/(norm(a)*norm(b))

def l2(a,b):
    x = np.asarray(a)-np.asarray(b)
    return norm(x)

model = OpenAIEmbeddings(model='text-embedding-ada-002')
#oc_embeddings = OpenAIEmbeddings(model='text-embedding-ada-002')

query = "国际争端"
documents = [
    "联合国就苏丹达尔富尔地区大规模暴力事件发出警告",
    "土耳其、芬兰、瑞典与北约代表将继续就瑞典“入约”问题进行谈判",
    "日本岐阜市陆上自卫队射击场内发生枪击事件 3人受伤",
    "国家游泳中心（水立方）：恢复游泳、嬉水乐园等水上项目运营",
    "我国首次在空间站开展舱外辐射生物学暴露实验",
]

query_vec = model.embed_query(query)
doc_vecs = model.embed_documents(documents)

print("Cosine distance:") #越大越相似
print(cos_sim(query_vec,query_vec))
for vec in doc_vecs:
    print(cos_sim(query_vec,vec))

print("\nEuclidean distance:") #越小越相似
print(l2(query_vec,query_vec))
for vec in doc_vecs:
    print(l2(query_vec,vec))

Cosine distance:
1.0
0.7806203619322558
0.7926144228096119
0.7616280759223305
0.7182043590648329
0.7306356855118128

Euclidean distance:
0.0
0.662389062540602
0.6440272888198665
0.6904663943141007
0.7507271636300279
0.7339813496648758


### 基于相似度聚类

In [13]:
from langchain.embeddings import OpenAIEmbeddings
from sklearn.cluster import KMeans, DBSCAN
import numpy as np

texts = [
    "这个多少钱",
    "啥价",
    "给我报个价",
    "我要红色的",
    "不要了",
    "算了",
    "来红的吧",
    "作罢",
    "价格介绍一下",
    "红的这个给我吧"
]
model = OpenAIEmbeddings(model='text-embedding-ada-002')
X = []
for t in texts:
    embedding = model.embed_query(t)
    X.append(embedding)
    
#clusters = KMeans(n_clusters=3, random_state=42, n_init="auto").fit(X)
clusters = DBSCAN(eps=0.55, min_samples=2).fit(X)
for i,t in enumerate(texts):
    print("{}\t{}".format(clusters.labels_[i],t))

0	这个多少钱
0	啥价
0	给我报个价
1	我要红色的
-1	不要了
-1	算了
1	来红的吧
-1	作罢
0	价格介绍一下
1	红的这个给我吧
